# Mini-Data Set Preparation

After the Kaggle Script "Making a mini-data set" is run (FYI, it takes about 2 minutes to run) to reduce the size of the data to 40,000 instances, run this script to organize data into a single dataframe. 

Run this with the 8 csv files produced by the Kaggle Script in the same directory. 

Note: This is a Python3 script because that is what Kaggle uses. 

In [203]:
import pandas as pd
import numpy as np
import copy
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [33]:
clicks_train_og = pd.read_csv("../input/clicks_train.csv")
promoted_content_og = pd.read_csv("../input/promoted_content.csv")
doc_cats_og = pd.read_csv("../input/documents_categories.csv")
doc_ents_og = pd.read_csv("../input/documents_entities.csv")
doc_meta_og = pd.read_csv("../input/documents_meta.csv")
doc_topics_og = pd.read_csv("../input/documents_topics.csv")
events_og = pd.read_csv("../input/events.csv")
page_views_og = pd.read_csv("../input/page_views_sample.csv")

/Users/nicolelee/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
# FOR TESTING WITH MINI DATASET

doc_ids = set(page_views_og['document_id']) & set(promoted_content_og['document_id'])
# pull in the content that is in both page_views and promoted_content

events = events_og[events_og['document_id'].isin(doc_ids)]
clicks_train = clicks_train_og[clicks_train_og['display_id'].isin(events['display_id'])]

events = events[events['display_id'].isin(clicks_train['display_id'])]

promoted_content = promoted_content_og[promoted_content_og['ad_id'].isin(clicks_train['ad_id'])]
doc_cats = doc_cats_og[doc_cats_og['document_id'].isin(promoted_content['document_id'])]
doc_ents = doc_ents_og[doc_ents_og['document_id'].isin(promoted_content['document_id'])]
doc_meta = doc_meta_og[doc_meta_og['document_id'].isin(promoted_content['document_id'])]
doc_topics = doc_topics_og[doc_topics_og['document_id'].isin(promoted_content['document_id'])]
page_views = page_views_og[page_views_og['document_id'].isin(events['document_id'])]

In [52]:
# FOR FULL DATASET ON AWS

display_sample = np.random.choice(clicks_train_og["display_id"].unique(), 10000) # change this if too many rows
clicks_train = clicks_train_og[clicks_train_og["display_id"].isin(display_sample)]
# select 4000 random display id's and grab all rows in click_train with that display
# every display has multiple ads and only 1 ad in every display is clicked
promoted_content = promoted_content_og[promoted_content_og["ad_id"].isin(clicks_train["ad_id"])]
# same ad can show up in multiple displays, so length of unique ads < length of unique displays
doc_cats = doc_cats_og[doc_cats_og["document_id"].isin(promoted_content["document_id"])]
doc_ents = doc_ents_og[doc_ents_og["document_id"].isin(promoted_content["document_id"])]
doc_meta = doc_meta_og[doc_meta_og["document_id"].isin(promoted_content["document_id"])]
doc_topics = doc_topics_og[doc_topics_og["document_id"].isin(promoted_content["document_id"])]
events = events_og[events_og["display_id"].isin(clicks_train_og["display_id"])]
page_views = page_views_og[page_views_og["document_id"].isin(promoted_content["document_id"])]
# platform & traffic source need to be either all integers or all strings (right now its mixed)


## Make master data merging all features to clicks_train

### Merge information about the displays to master dataset
Events are only if the user CLICKED.  
Page views are all logs of users visiting the page.

In [110]:
data = clicks_train.merge(events, on='display_id', how='left')
data.head(20)

# joins information about the display that the user saw
# each display has a unique user id, doc id, and timestamp
# events has the information about the display (who the user is, which site (document_id) it was on, when it was seen, from where, etc.)

,display_id,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location
0,37,70153,0,d4f62cdcb39ad8,1779285,2687,2,US>WA>819
1,37,149047,0,d4f62cdcb39ad8,1779285,2687,2,US>WA>819
2,37,169564,0,d4f62cdcb39ad8,1779285,2687,2,US>WA>819
3,37,234713,1,d4f62cdcb39ad8,1779285,2687,2,US>WA>819
4,37,235443,0,d4f62cdcb39ad8,1779285,2687,2,US>WA>819
5,101,10731,0,3abf1630ff91c5,1029942,7396,1,CA>BC
6,101,17917,0,3abf1630ff91c5,1029942,7396,1,CA>BC
7,101,18493,0,3abf1630ff91c5,1029942,7396,1,CA>BC
8,101,51402,1,3abf1630ff91c5,1029942,7396,1,CA>BC
9,101,117307,0,3abf1630ff91c5,1029942,7396,1,CA>BC


### Merge information about the documents the ads refer to
All the doc files have information about the documents (websites) to which the ads refer to
including confidence levels of which topics the ads referred to, which categories they're apart of, etc.

We wanted to duplicate the idea of the CountVectorizer for the 'bag of words' model we used for spam detection, but since we're not counting words in a text, it's a little bit different. Since we have a 'dictionary' of categories and topics, we use that as our 'vocabulary.' Every document has a confidence level for one or more items in the vocabulary, so we create a sparse matrix with every topic and category as columns, and every document has a confidence level value in the respective columns. If they are not given a confidence level, we put 0 because the document most likely does not have anything to do with that category or topic (given the data provided by Outbrain).

This data on the documents will help us separate ads from one another based on topic/category.  
ie) why did ad A get clicked instead of ad B? We know ad A referred to document 1 whereas ad B referred to document 2, and now we have general information about the documents the ads referred to. We will merge this information in later steps.

In [227]:
sparsetop = doc_topics.pivot(index='document_id', columns='topic_id', values='confidence_level')
sparsetop.columns = ['top_' + str(col) for col in sparsetop.columns]

sparsecat = doc_cats.pivot(index='document_id', columns='category_id', values='confidence_level')
sparsecat.columns = ['cat_' + str(col) for col in sparsecat.columns]

sparse = sparsetop.join(sparsecat, how='outer')
sparse.fillna(0, inplace=True)

In [230]:
promoted_content

,ad_id,document_id,campaign_id,advertiser_id
38,39,471548,5,19
64,65,29861,26,39
162,164,39820,28,45
369,371,13719,28,45
478,480,64398,37,63
485,487,941078,81,116
487,489,941048,81,116
518,520,81487,37,63
589,591,91275,53,86
593,596,103318,37,63


In [8]:
#there is not a one-to-one relationship between document_id in promoted and the master data
#This is because the same ad is being shown in different documents I think
print(len(promoted["document_id"].unique()))
print(len(data["document_id"].unique()))

4230
7240


In [9]:
#there is a one-to-one relationship between ad_id in promoted and the master data
print(len(promoted["ad_id"].unique())) #each add can appear more than once
print(len(data["ad_id"].unique()))

6042
6042


In [10]:
promoted.head()
del promoted["Unnamed: 0"]
del promoted['document_id'] #I think all we want from here is the link between ad_id and campaign id
promoted.head()

,ad_id,campaign_id,advertiser_id
0,489,81,116
1,844,37,63
2,971,28,45
3,1117,205,134
4,1126,209,295


## Joining Info about each ad

I make a dictionary of the advertiser and campaign id for each ad_id, map that dictionary to the ad id to make the advertizer and campain columns

In [19]:
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location
display_id,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602


In [12]:
print(len(data))
print(len(data["ad_id"].unique())) #adds appear on average slightly more than twice in our minidata set

10000
6042


In [13]:
#make dictionaries to look up advertizer id and campaign id for each ad_id
advertiser_dict = dict(zip(promoted.ad_id, promoted.advertiser_id))
campaign_dict = dict(zip(promoted.ad_id, promoted.campaign_id))


In [14]:
data["campaign_id"] = data["ad_id"].map(campaign_dict)
data["advertiser_id"] = data["ad_id"].map(advertiser_dict)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id
display_id,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876


In [15]:
print(len(data))
print(len(data["ad_id"].unique())) #adds appear on average slightly more than twice in our minidata set

10000
6042


## Working with Page Views

Add count of page views to every document in data as a feature, could tell us something about likelihood of ads being clicked

In [61]:
print(len(page_views))
print(len(page_views['document_id'].unique()))
# when we filter our data like this, we only get 23 documents from page_views_sample.. we need the full dataset


36071
23


## Importing Document Information

I'm super stuck on why all the document ids that appear in our data arent in the files with more information about each documents.

In [16]:
#Why aren't there the same number of unique documents in each of these
print(len(data["document_id"].unique()))
print(len(doc_cats["document_id"].unique()))
print(len(doc_ents["document_id"].unique()))
print(len(doc_meta["document_id"].unique()))
print(len(doc_topics["document_id"].unique()))

7240
4223
3540
4230
4154


In [7]:
#each document has multiple possible entities, categories, topics with different confidence level. 
#maybe we should just for now keep the most likely entity, topic and category? 
doc_ents.head()

,document_id,entity_id,confidence_level
72655,1807706,11d5279de0b36011773ee50f75f9b43c,0.495961
72656,1807706,358c9e8a2ff80f858f6deb063e0bcf8f,0.337521
72657,1807706,b2a032747f34ec8488b2be2ebe180d9e,0.281354
72673,1316839,e8493cde7d253c7def5b030a2c94357d,0.977918
72674,1316839,c5039d9dda9bccb8ed150d8affedd306,0.375150


In [18]:
doc_cats.head()

,Unnamed: 0,document_id,category_id,confidence_level
0,51976,1807706,1209,0.92
1,51977,1807706,1205,0.07
2,51990,1316839,2003,0.92
3,51991,1316839,1211,0.07
4,51994,479029,1205,0.92


In [55]:
data[data['document_id']==1405372]

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location
display_id,,,,,,,
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616


In [20]:
data["geo_location"] = data["geo_location"].apply(str)

In [21]:
data["country"] = [x[:2] for x in data["geo_location"]]
data["state"] = [x[3:5] for x in data["geo_location"]]
data["loc_num"]= [x[6:] for x in data["geo_location"]]
data["loc_num"] = data["loc_num"].map(str)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,state,loc_num
display_id,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,MI,563
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,MO,616
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,GA,524
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,CA,807
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,IL,602


In [22]:
list_to_binarize = ["advertiser_id", "campaign_id", "platform", "country"]

for i in list_to_binarize:
    temp = pd.get_dummies(data[i], prefix = i)
    data = data.join(temp)
data.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,...,0,0,0,1,0,0,0,0,0,0
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,...,0,0,0,1,0,0,0,0,0,0
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,...,0,0,0,1,0,0,0,0,0,0
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,...,0,0,0,1,0,0,0,0,0,0
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,...,0,0,0,1,0,0,0,0,0,0


In [34]:
clean = copy.deepcopy(data)
clean.head()

,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,campaign_id,advertiser_id,country,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,151682,1,715ddc85c78e39,1792396,559250,2,US>MI>563,19284,3283,US,...,0,0,0,1,0,0,0,0,0,0
10333,157598,0,ec3841c3250fb6,1405372,716674,2,US>MO>616,18022,2312,US,...,0,0,0,1,0,0,0,0,0,0
11278,103712,1,324475e5f191c8,1179111,784022,2,US>GA>524,5081,1759,US,...,0,0,0,1,0,0,0,0,0,0
13320,139944,1,27221a85141108,735143,926961,2,US>CA>807,17929,1782,US,...,0,0,0,1,0,0,0,0,0,0
13905,173393,0,9ea0907cee579e,1794259,971672,2,US>IL>602,21143,2876,US,...,0,0,0,1,0,0,0,0,0,0


In [35]:
len(data["ad_id"].unique())

6042

In [36]:
len(clean['document_id'].unique())

7240

In [37]:
#drop the non-cleaned up data for now
del clean['country'], clean['state'], clean["ad_id"], clean["uuid"], clean["document_id"],clean["timestamp"], clean["platform"], clean["geo_location"], clean['campaign_id'], clean["advertiser_id"]

In [39]:
len(clean.columns)
clean.head()

,clicked,loc_num,advertiser_id_4,advertiser_id_5,advertiser_id_8,advertiser_id_9,advertiser_id_10,advertiser_id_12,advertiser_id_13,advertiser_id_14,...,country_TZ,country_UA,country_UG,country_US,country_UZ,country_VI,country_VN,country_ZA,country_ZM,country_ZW
display_id,,,,,,,,,,,,,,,,,,,,,
8112,1,563,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
10333,0,616,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
11278,1,524,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13320,1,807,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
13905,0,602,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Make training and test sets

In [45]:
labels = clean['clicked']
labels = labels.values.reshape(-1,1) # check this please! my python is 3.5 and told me to use values.reshape
del clean['clicked']

In [55]:
train_data = clean[:round(.7*len(clean))]
train_labels = labels[:round(.7*len(clean))]
test_data = clean[round(.7*len(clean)):]
test_labels = labels[round(.7*len(clean)):]

In [56]:
lr = LogisticRegression()
lr.fit(train_data, train_labels)
lr.predict(test_data, test_labels)

# Logistic Regression is not the best model because we have to binarize our features
# to make them numeric. This leads to >5000 columns which is unruly and not optimal.

ValueError: could not convert string to float: 